In [ ]:
import openai
import sqlite3
import pandas as pd
import os
import re
import time
import random
from openai.error import RateLimitError, OpenAIError

# OpenAI API key
openai.api_key = "Your API here"


# Helper Function

In [3]:
def generate_sql(user_question, model="ft:gpt-4o-2024-08-06:personal::AXYv83vn", max_retries=5):
    """
    Converts a natural language question into an SQL query using a fine-tuned GPT chat model.
    
    Implements a retry mechanism with exponential backoff to handle rate limit errors.
    
    :param user_question: The user's natural language question
    :param model: The name of the fine-tuned GPT chat model
    :param max_retries: Maximum number of retry attempts
    :return: The generated SQL query as a string
    """
    prompt = f"""
Pretend you are an expert at converting natural language questions into accurate SQL queries. Please generate an accurate SQL query based on the following natural language question and database schema provided below. Think sequentially and refer to the sample natural language questions with correct and incorrect outputs as well.

Database Schema:
Table 1: t_zacks_fc (This table contains fundamental indicators for companies)
Columns: 'ticker' = Unique zacks Identifier for each company/stock, ticker or trading symbol, 'comp_name' = Company name, 'exchange' = Exchange traded, 'per_end_date' = Period end date which represents quarterly data, 'per_type' = Period type (eg. Q for quarterly data), 'filing_date' = Filing date, 'filing_type' = Filing type: 10-K, 10-Q, PRELIM, 'zacks_sector_code' = Zacks sector code (Numeric Value eg. 11 = Aerospace), 'eps_diluted_net_basic’ = Earnings per share (EPS) net (Company's net earnings or losses attributable to common shareholders per basic share basis), 'lterm_debt_net_tot' = Net long-term debt (The net amount of long term debt issued and repaid. This field is either calculated as the sum of the long term debt fields or used if a company does not report debt issued and repaid separately).
Keys: ticker, per_end_date, per_type

Table 2: t_zacks_fr (This table contains fundamental ratios for companies)
Columns: 'ticker' = Unique zacks Identifier for each company/stock, ticker or trading symbol, 'per_end_date' = Period end date which represents quarterly data, 'per_type' = Period type (eg. Q for quarterly data), ‘ret_invst’ = Return on investments (An indicator of how profitable a company is relative to its assets invested by shareholders and long-term bond holders. Calculated by dividing a company's operating earnings by its long-term debt and shareholders equity), ‘tot_debt_tot_equity’ = Total debt / total equity (A measure of a company's financial leverage calculated by dividing its long-term debt by stockholders' equity).
Keys: ticker, per_end_date, per_type.

Table 3: t_zacks_mktv (This table contains market value data for companies)
Columns: 'ticker' = Unique zacks Identifier for each company/stock, ticker or trading symbol, 'per_end_date' = Period end date which represents quarterly data, 'per_type' = Period type (eg. Q for quarterly data), ‘mkt_val’ = Market Cap of Company (shares out x last monthly price per share - unit is in Millions).
Keys: ticker, per_end_date, per_type.

Table 4: t_zacks_shrs (This table contains shares outstanding data for companies)
Columns: 'ticker' = Unique zacks Identifier for each company/stock, ticker or trading symbol, 'per_end_date' = Period end date which represents quarterly data, 'per_type' = Period type (eg. Q for quarterly data), ‘shares_out’ = Number of Common Shares Outstanding from the front page of 10K/Q.
Keys: ticker, per_end_date, per_type.

Table 5: t_zacks_sectors (This table contains the zacks sector codes and their corresponding sectors)
Columns: 'zacks_sector_code' = Unique identifier for each zacks sector, 'sector': the sector descriptions that correspond to the sector code 
Keys: zacks_sector_code 

Sample natural language questions with correct and incorrect outputs: 
Sample prompt 1: Output ticker with the largest market value recorded on any given period end date. 
Correct output for prompt 1: SELECT ticker, per_end_date, MAX(mkt_val) AS max_market_value FROM t_zacks_mktv GROUP BY per_end_date ORDER BY max_market_value DESC LIMIT 1;
Incorrect output for prompt 1: SELECT MAX(mkt_val) , ticker FROM t_zacks_mktv GROUP BY ticker

Sample prompt 2: What is the company name with the lowest market cap?
Correct output for prompt 2: SELECT fc.comp_name, mktv.ticker, mktv.mkt_val FROM t_zacks_mktv AS mktv JOIN t_zacks_fc AS fc ON mktv.ticker = fc.ticker WHERE mktv.mkt_val = (SELECT MIN(mkt_val) FROM t_zacks_mktv);
Incorrect output for prompt 2:  SELECT T1.comp_name FROM t_zacks_fc AS T1 INNER JOIN t_zacks_mktv AS T2 ON T1.ticker = T2.ticker AND T1.per_end_date = T2.per_end_date AND T1.per_type = T2.per_type ORDER BY T2.mkt_val LIMIT 1

Sample prompt 3: Filter t_zacks_fc to only show companies with a total debt-to-equity ratio greater than 1.
Correct output for prompt 3: SELECT * FROM t_zacks_fr WHERE tot_debt_tot_equity > 1;
Incorrect output for prompt 3: SELECT * FROM t_zacks_fr WHERE t_zacks_mktv > 1;

Sample prompt 4: Filter t_zacks_shrs to include companies with more than 500 million shares outstanding as of the most recent quarter.
Correct output for prompt 4: SELECT *
FROM t_zacks_shrs
WHERE shares_out > 5000
ORDER BY per_end_date DESC;
Incorrect output for prompt 4: SELECT * FROM t_zacks_shrs WHERE shares_out > 500000000

Sample prompt 5: Combine t_zacks_mktv and t_zacks_shrs to show tickers with market cap and shares outstanding in the latest period end date.
Correct output for prompt 5: SELECT mktv.ticker, mktv.per_end_date, mktv.mkt_val, shrs.shares_out
FROM t_zacks_mktv mktv
JOIN t_zacks_shrs shrs ON mktv.ticker = shrs.ticker AND mktv.per_end_date = shrs.per_end_date
ORDER BY mktv.per_end_date DESC;
Incorrect output for prompt 5: SELECT ticker, mkt_val, shares_out FROM t_zacks_mktv INNER JOIN t_zacks_shrs ON t_zacks_mktv.ticker = t_zacks_shrs.ticker AND t_zacks_mktv.per_end_date = t_zacks_shrs.per_end_date ORDER BY per_end_date DESC LIMIT 1

Sample prompt 6: Join t_zacks_fc and t_zacks_fr to show tickers with total debt-to-equity ratios and EPS from NASDAQ as of Q2 2024.
Correct output for prompt 6: SELECT fc.ticker, fc.eps_diluted_net_basic, fr.tot_debt_tot_equity
FROM t_zacks_fc fc
JOIN t_zacks_fr fr ON fc.ticker = fr.ticker AND fc.per_end_date = fr.per_end_date
WHERE fc.exchange = 'NASDAQ' AND fc.per_type = 'Q' AND fc.per_end_date BETWEEN '2024-04-01' AND '2024-06-30';
Incorrect output for prompt 6: SELECT T1.ticker, T1.eps_diluted_net_basic, T2.ret_invst, T2.tot_debt_tot_equity FROM t_zacks_fc AS T1 INNER JOIN t_zacks_fr AS T2 ON T1.ticker = T2.ticker AND T1.per_end_date = T2.per_end_date WHERE T1.exchange = 'NASDAQ' AND T1.per_type = 'Q2';

Please make sure that when you are joining 2 or more tables, you are using all 3 keys (ticker, per_end_date & per_type). Also, ensure that the SQL query is syntactically correct and provides the expected output based on the natural language question provided.

User's Question:
{user_question}

Please provide only the SQL query without any markdown, code block syntax, or explanations.
    """
    attempt = 0
    while attempt < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300,  # Adjust based on complexity
                temperature=0.0,  # Set to 0 for deterministic output
                n=1,
                stop=None
            )
            # Retrieve the generated SQL query
            raw_sql = response.choices[0].message['content'].strip()
            print("\nGenerated SQL Query:")
            print(raw_sql)

            # Clean the SQL query by removing any markdown or code block syntax
            sql_query = re.sub(r'^```sql\s*', '', raw_sql, flags=re.IGNORECASE)
            sql_query = re.sub(r'```$', '', sql_query, flags=re.IGNORECASE)
            sql_query = sql_query.strip()

            print("\nCleaned SQL Query:")
            print(sql_query)
            return sql_query

        except RateLimitError as e:
            wait_time = 2 ** attempt + random.uniform(0, 1)
            print(f"Rate limit reached. Waiting for {wait_time:.2f} seconds before retrying...")
            time.sleep(wait_time)
            attempt += 1
        except OpenAIError as e:
            print(f"Error generating SQL query: {e}")
            return None

    print("Max retries exceeded. Failed to generate SQL query.")
    return None

def execute_sql(sql, database):
    """
    Executes an SQL query against a SQLite database.

    :param sql: The SQL query to execute
    :param database: Path to the SQLite database file
    :return: Result of the query as a Pandas DataFrame
    """
    try:
        with sqlite3.connect(database) as conn:
            result = pd.read_sql_query(sql, conn)
        return result
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        return None




# Main Function

In [ ]:
# Define the path to your SQLite database
database = r"E:\【Chicago】\[Autumn Semester\Project Lab\Database\zack_database.db"

# Define a list of natural language prompts
df_list = pd.read_excel('BofA Project Lab - ChatGPT New Prompts.xlsx')
list_prompts = df_list['Prompt'].values.tolist()

# Initialize dictionaries to store results
sql_dict = {}
result_dict = {}
output_results = []

for i, prompt in enumerate(list_prompts):
    print(f"\nProcessing Prompt {i+1}: {prompt}")
    
    # Generate SQL query from the prompt
    sql_query = generate_sql(prompt)
    if not sql_query:
        result_dict[i] = None
        output_results.append("SQL Generation Failed")
        continue
    
    sql_dict[i] = sql_query
    
    # Execute the SQL query on the SQLite database
    result = execute_sql(sql_query, database)
    result_dict[i] = result
    
    if result is not None and not result.empty:
        if result.shape == (1, 1):
            output_results.append(result.iloc[0, 0])
        else:
            output_results.append(result.shape)
    else:
        output_results.append("No Results Found or Execution Failed")

# Create a DataFrame to consolidate the results
df = pd.DataFrame({
    'Prompt': list_prompts,
    'SQL Query': [sql_dict.get(i, "") for i in range(len(list_prompts))],
    'Query Result': output_results
})




Processing Prompt 1: What is the highest quarterly return on investment value for any ticker in 2006 ?

Generated SQL Query:
SELECT ticker, MAX(ret_invst) AS max_roi FROM t_zacks_fr WHERE per_type = 'Q' AND per_end_date BETWEEN '2006-01-01' AND '2006-12-31' GROUP BY ticker ORDER BY max_roi DESC LIMIT 1;

Cleaned SQL Query:
SELECT ticker, MAX(ret_invst) AS max_roi FROM t_zacks_fr WHERE per_type = 'Q' AND per_end_date BETWEEN '2006-01-01' AND '2006-12-31' GROUP BY ticker ORDER BY max_roi DESC LIMIT 1;

Processing Prompt 2: What is the ticker for the company with the lowest shares outstanding in 2020?

Generated SQL Query:
SELECT ticker, MIN(shares_out) AS min_shares_out FROM t_zacks_shrs WHERE per_end_date BETWEEN '2020-01-01' AND '2020-12-31' GROUP BY ticker ORDER BY min_shares_out ASC LIMIT 1;

Cleaned SQL Query:
SELECT ticker, MIN(shares_out) AS min_shares_out FROM t_zacks_shrs WHERE per_end_date BETWEEN '2020-01-01' AND '2020-12-31' GROUP BY ticker ORDER BY min_shares_out ASC LIMIT 

In [5]:
# Display the DataFrame
df

,Prompt,SQL Query,Query Result
0,What is the highest quarterly return on invest...,"SELECT ticker, MAX(ret_invst) AS max_roi FROM ...",ticker max_roi 0 OPK 1191386.0
1,What is the ticker for the company with the lo...,"SELECT ticker, MIN(shares_out) AS min_shares_o...",ticker min_shares_out 0 GPUS 0.0
2,What is the total debt to total equity ratio f...,SELECT tot_debt_tot_equity FROM t_zacks_fr WHE...,1.3192
3,What stock exchange was the company Lam Resear...,SELECT exchange FROM t_zacks_fc WHERE comp_nam...,NASDAQ
4,Output all the company tickers and sector code...,"SELECT fc.ticker, fc.zacks_sector_code\nFROM t...",ticker zacks_sector_code 0 DBI ...
...,...,...,...
72,Find the total number of companies with market...,SELECT COUNT(*) AS company_count\nFROM (\n ...,148
73,Calculate the sum of shares outstanding for co...,SELECT SUM(shrs.shares_out) AS total_shares_ou...,29343.23
74,Determine the ticker of the company with the l...,"SELECT fr.ticker, fr.ret_invst\nFROM t_zacks_f...",ticker ret_invst 0 AEMD -8255.738
75,Output the tickers of the companies in the ban...,SELECT mktv.ticker\nFROM t_zacks_mktv AS mktv\...,ticker 0 AFL 1 AIG 2 ALL 3 A...


In [6]:
df.to_excel('testing_GPT_SQL.xlsx')

In [9]:
result.shape

(59, 14)